In [1]:
import sys
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import os

sys.path.append("kaggle/input/neurips-open-polymer-prediction-2025")
base_path = "kaggle/input/neurips-open-polymer-prediction-2025/"
supplement_path = "kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/"
tc_smiles_path = "kaggle/input/tc-smiles/"
smiles_extra_data_path = "kaggle/input/smiles-extra-data/"

In [3]:
import torch
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
import optuna

from data_preparation import load_qm9_dataset
from train import pretrain_qm9


In [4]:
from torch_geometric.datasets import QM9

dataset = QM9(root="data/qm9")
print(dataset)

Extracting data\qm9\raw\qm9.zip
Processing...
100%|██████████| 133885/133885 [01:08<00:00, 1952.98it/s]
Done!


QM9(130831)


In [6]:
# -----------------------------
# 环境设置
# -----------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Training on {device}")

✅ Training on cpu


In [7]:
# -----------------------------
# 数据加载
# -----------------------------
loader, dataset = load_qm9_dataset(root="data/qm9", batch_size=8172)
n = len(dataset)
train_set, val_set = random_split(dataset, [int(0.9*n), n - int(0.9*n)])
train_loader = DataLoader(train_set, batch_size=256, shuffle=True)
val_loader = DataLoader(val_set, batch_size=256)

In [8]:
print(dataset.num_node_features, dataset.num_edge_features)

11 4


In [ ]:
# -----------------------------
# Optuna 目标函数
# -----------------------------
def objective(trial):
    ckpt_path = f"checkpoints/trial_{trial.number}_best.pt"
    model, hist = pretrain_qm9(
        train_loader=train_loader,
        val_loader=val_loader,
        dataset=dataset,
        hidden_dim=trial.suggest_categorical("hidden_dim", [128, 256, 512, 1024]),
        lr=trial.suggest_loguniform("lr", 1e-4, 1e-2),
        num_edge_layers=trial.suggest_int("num_edge_layers", 2, 16),
        dropout=trial.suggest_float("dropout", 0.0, 0.3),
        epochs=20,
        device=device,
        run_wandb=False,
        ckpt_path=ckpt_path
    )
    return min(hist["val_loss"])

In [1]:
import optuna

storage_path = "sqlite:///optuna_qm9.db"  # 保存在当前目录
study = optuna.create_study(
    direction="minimize",
    study_name="qm9_pretrain",
    storage=storage_path,
    load_if_exists=True
)

d:\anaconda\envs\neurips\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-09-09 00:03:47,148] Using an existing study with name 'qm9_pretrain' instead of creating a new one.


In [ ]:
# -----------------------------
# 运行超参搜索
# -----------------------------
study.optimize(objective, n_trials=30)

print("✅ 最优超参数:", study.best_trial.params)
print("📉 最优验证集 loss:", study.best_value)

In [3]:
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_rank
from optuna.visualization import plot_slice
from optuna.visualization import plot_timeline

In [4]:
plot_optimization_history(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'mode': 'markers',
              'name': 'Objective Value',
              'type': 'scatter',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
              'y': [1590.8214205228364, 1814.6787954477163, 1654.2633596567007,
                    1845.354252741887, 1652.5073570838342, 1477.7825481708232,
                    1387.6915658804087, 1551.9534536508413, 1597.812720665565,
                    1455.884537916917, 1352.9065786508413, 1361.9598717322717,
                    1322.3310335599458, 1314.2560190054087, 1675.2108013446514,
                    1744.5076012244592, 1653.798349233774, 1429.6073420597957,
                    1516.90037888747, 1501.4687194824219, 1415.3237163837139,
                    1300.323223407452, 1535.5472318209136, 1605.04830463116,
                    1380.9502962552583, 1538.7987717848557, 1471.225588285006,
                    1306.7744481013372, 1637.5396282489482, 1634.290787916917]},
             {'mode': 'lines',
              'name': 'Best Value',
              'type': 'scatter',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
              'y': [1590.8214205228364, 1590.8214205228364, 1590.8214205228364,
                    1590.8214205228364, 1590.8214205228364, 1477.7825481708232,
                    1387.6915658804087, 1387.6915658804087, 1387.6915658804087,
                    1387.6915658804087, 1352.9065786508413, 1352.9065786508413,
                    1322.3310335599458, 1314.2560190054087, 1314.2560190054087,
                    1314.2560190054087, 1314.2560190054087, 1314.2560190054087,
                    1314.2560190054087, 1314.2560190054087, 1314.2560190054087,
                    1300.323223407452, 1300.323223407452, 1300.323223407452,
                    1300.323223407452, 1300.323223407452, 1300.323223407452,
                    1300.323223407452, 1300.323223407452, 1300.323223407452]},
             {'marker': {'color': '#cccccc'},
              'mode': 'markers',
              'name': 'Infeasible Trial',
              'showlegend': False,
              'type': 'scatter',
              'x': [],
              'y': []}],
    'layout': {'template': '...',
               'title': {'text': 'Optimization History Plot'},
               'xaxis': {'title': {'text': 'Trial'}},
               'yaxis': {'title': {'text': 'Objective Value'}}}
})

In [ ]:
plot_slice(study)